In [2]:
from openai import OpenAI
from load_api_key import load_openai_api_key

key = load_openai_api_key()
client = OpenAI(api_key=key)

API Key loaded successfully.


In [3]:
assistant = client.beta.assistants.create(
    name="ML Expert",
    instructions="You are a machine learning expert. When asked a question, write and run code to answer the question.",
    model="gpt-4o-mini",
    tools=[{"type":"code_interpreter"}]
    # too_resources to provide file
)

In [4]:
thread = client.beta.threads.create()

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want to create a simple python program that prints 'Hello world' to the terminal."
)

In [6]:
from typing_extensions import override
from openai import AssistantEventHandler

# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

# Then, we use the `stream` SDK helper
# with the `EventHandler` class to create the Run
# and stream the response.

with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > Here's a simple Python program that prints "Hello, World!" to the terminal:

```python
print("Hello, World!")
```

You can run this code in any Python environment. Would you like help with anything else, Jane Doe?